IMPORTAÇÃO DE MODULOS

In [1]:
from path_arquivos import *
import pandas as pd
import numpy as np
import glob
import os

EXTRAÇÃO

In [2]:
acum = pd.read_excel(bi_abst.cons_64)
caminho = r'c:\WS_OLIVEIRA\BASE_DADOS\BASE_8664\10 - OUTUBRO 8664.xlsx'
mes_10 = pd.read_excel(caminho)

c:\WS_OLIVEIRA\SCRIPTS\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


CORPO DO CODIGO

In [20]:
mes_10['DATAGERACAO'] = pd.to_datetime(mes_10['DATAGERACAO']).dt.normalize()
mes_10['MES'] = mes_10['DATAGERACAO'].dt.month
mes_10 = mes_10[['DATAGERACAO', 'DTLANC', 'NUMBONUS', 'NUMOS', 'CODEMPILHADOR','EMPILHADOR', 'MES']].copy()
mes_10 = mes_10.drop_duplicates(subset=['NUMOS'])

novo_ciclo = pd.concat([acum, mes_10], ignore_index= True)
novo_ciclo = novo_ciclo.loc[novo_ciclo['MES'].astype(float) > 0]

CONSULTAS

In [21]:
grupo = novo_ciclo.groupby('MES').agg(
    qtde = ('MES', 'count')
).reset_index()
display(grupo.columns)
display(grupo.sort_values(by='MES', ascending= False))

Index(['MES', 'qtde'], dtype='object')

,MES,qtde
6,11.0,414
5,10.0,43812
4,9.0,29086
3,8.0,27556
2,7.0,37137
1,6.0,29194
0,5.0,37527


In [22]:
novo_ciclo.to_excel(bi_abst.cons_64, index= False, sheet_name= '8664')